In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [84]:
#  This function finds the percentage of each value compared to the previous one
#  For the first value, since there is not a previous one, I find the difference between the first element and the second one
def AvgPrevious(nparray):
    for i in range(len(nparray)):
        if i > 0:
            nparray[i] = (nparray[i] - nparray[i - 1])/nparray[i]
        else:
            nparray[i] = (nparray[i] - nparray[i + 1])/nparray[i]
    return nparray

In [124]:
# Trying to convert the input data, which starts off as dataframeX, must take 5 days and then transpose it
# and then convert it into a numpy array

rawdata = pd.read_csv('data/bitcoin.csv',usecols=[8])
days = 5                                                      # Number of days we are looking at
predict_delta = 7                                             # Number of days we are predicting into the future 
number_of_items = rawdata.shape[0] - (predict_delta + days)   # number of tests
print(rawdata.shape)

X = []                                                        # five day blocks in a list
# For every index in x, there is a y that holds the actual 
# value that we are trying to predict (seven days into the future)
Y = []
#Y = rawdata.tail(number_of_items).as_matrix().tolist()

# Splits the array of closing prices up into multiple arrays, each holding five days worth of prices
# then transposes each array and finally appends them onto one large matrix
for i in range(number_of_items - 1000): 
    predict_day = rawdata.head(days + i + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    last_day = rawdata.head(days + i).tail(1).as_matrix().transpose().tolist()[0]
    
    # Adding data to arrays
    X.append(AvgPrevious(rawdata.head(days).tail(days).as_matrix().transpose().tolist()[0]))
    Y.append((predict_day - last_day[0])/predict_day)

# converts python arrays into numpy arrays
X = np.array(X)
Y = np.array(Y)

test = AvgPrevious(rawdata.head(days).tail(days).as_matrix().transpose().tolist()[0])
print(test)



(1745, 1)
[-0.07696892928991866, 1.0005325095426174, 0.9928019243917798, 0.9915137881494847, 0.9905758598217899]


In [125]:
model = Sequential()
model.add(Dense(1,input_shape=(5,)))
model.add(Activation('relu'))
model.add(Dense(1, activation='tanh'))
model.add(Dropout(1.0))

sgd = SGD(lr=0.000001)

model.compile(loss='mse',optimizer='rmsprop', metrics=['accuracy'])

H = model.fit(X, Y, epochs=10)

Epoch 1/10
733/733 [==============================] - 1s 2ms/step - loss: 0.0180 - acc: 0.0014
Epoch 2/10
733/733 [==============================] - 0s 101us/step - loss: 0.0179 - acc: 0.0014
Epoch 3/10
733/733 [==============================] - 0s 98us/step - loss: 0.0180 - acc: 0.0014
Epoch 4/10
733/733 [==============================] - 0s 127us/step - loss: 0.0180 - acc: 0.0014
Epoch 5/10
733/733 [==============================] - 0s 105us/step - loss: 0.0180 - acc: 0.0014
Epoch 6/10
733/733 [==============================] - 0s 113us/step - loss: 0.0180 - acc: 0.0014
Epoch 7/10
733/733 [==============================] - 0s 123us/step - loss: 0.0180 - acc: 0.0014
Epoch 8/10
733/733 [==============================] - 0s 100us/step - loss: 0.0180 - acc: 0.0014
Epoch 9/10
733/733 [==============================] - 0s 102us/step - loss: 0.0180 - acc: 0.0014
Epoch 10/10
733/733 [==============================] - 0s 104us/step - loss: 0.0180 - acc: 0.0014ETA: 0s - loss: 0.0179 - acc: 0.0

In [127]:
Test = AvgPrevious([8070.8, 8891.21, 8516.24, 9477.84, 10016.49])
XVal = np.array([Test])
r = model.predict(XVal)
result = r[0][0]

if result >= 1 - 2/3:
    print("it will go up by 3%")
elif result > 0 - 2/3 and result < 1 - 2/3:
    print("it will stay between 3%")
else:
    print("it will go down by 3%")

print(result)

it will go up by 3%
1.0


In [122]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_105 (Dense)            (None, 1)                 6         
_________________________________________________________________
activation_61 (Activation)   (None, 1)                 0         
_________________________________________________________________
dense_106 (Dense)            (None, 1)                 2         
_________________________________________________________________
dropout_55 (Dropout)         (None, 1)                 0         
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________
